In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from math import sqrt


In [4]:
df = pd.read_csv("C:/Users/chris/Documents/Senior_Project/CT_SeniorProject/CrashData_Repo1/data/interim/filtered_dataset.csv")
df_ord = pd.read_csv("C:/Users/chris/Documents/Senior_Project/CT_SeniorProject/CrashData_Repo1/data/external/ORD_Weather.csv")

#print(df.head())
print(df_ord.head())
df_ord.info()


#df_ord.shape()
df_ord.info()     # Overview of column types and missing values
df_ord.describe()   # Summary statistics for numeric columns
df_ord.isnull().sum()

       STATION                                        NAME  LATITUDE  \
0  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
1  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
2  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
3  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
4  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   

   LONGITUDE  ELEVATION        DATE   AWND AWND_ATTRIBUTES  PGTM  \
0  -87.93164      204.8  2018-01-01  10.74             ,,W   NaN   
1  -87.93164      204.8  2018-01-02  11.18             ,,W   NaN   
2  -87.93164      204.8  2018-01-03  12.30             ,,W   NaN   
3  -87.93164      204.8  2018-01-04  12.30             ,,W   NaN   
4  -87.93164      204.8  2018-01-05  10.74             ,,W   NaN   

  PGTM_ATTRIBUTES  ...  WT04 WT04_ATTRIBUTES  WT05 WT05_ATTRIBUTES  WT06  \
0             NaN  ...   NaN             NaN   NaN             NaN   NaN   
1     

STATION               0
NAME                  0
LATITUDE              0
LONGITUDE             0
ELEVATION             0
DATE                  0
AWND                  1
AWND_ATTRIBUTES       1
PGTM               2522
PGTM_ATTRIBUTES    2522
PRCP                  0
PRCP_ATTRIBUTES       0
SNOW                  0
SNOW_ATTRIBUTES       0
SNWD                  1
SNWD_ATTRIBUTES       1
TAVG                  0
TAVG_ATTRIBUTES       0
TMAX                  0
TMAX_ATTRIBUTES       0
TMIN                  0
TMIN_ATTRIBUTES       0
WDF2                  1
WDF2_ATTRIBUTES       1
WDF5                  6
WDF5_ATTRIBUTES       6
WSF2                  1
WSF2_ATTRIBUTES       1
WSF5                  6
WSF5_ATTRIBUTES       6
WT01               1550
WT01_ATTRIBUTES    1550
WT02               2492
WT02_ATTRIBUTES    2492
WT03               2235
WT03_ATTRIBUTES    2235
WT04               2522
WT04_ATTRIBUTES    2522
WT05               2535
WT05_ATTRIBUTES    2535
WT06               2516
WT06_ATTRIBUTES 

In [6]:
print(df_ord.head())
print(df_ord.info())

       STATION                                        NAME  LATITUDE  \
0  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
1  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
2  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
3  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   
4  USW00094846  CHICAGO OHARE INTERNATIONAL AIRPORT, IL US  41.96017   

   LONGITUDE  ELEVATION        DATE   AWND AWND_ATTRIBUTES  PGTM  \
0  -87.93164      204.8  2018-01-01  10.74             ,,W   NaN   
1  -87.93164      204.8  2018-01-02  11.18             ,,W   NaN   
2  -87.93164      204.8  2018-01-03  12.30             ,,W   NaN   
3  -87.93164      204.8  2018-01-04  12.30             ,,W   NaN   
4  -87.93164      204.8  2018-01-05  10.74             ,,W   NaN   

  PGTM_ATTRIBUTES  ...  WT04 WT04_ATTRIBUTES  WT05 WT05_ATTRIBUTES  WT06  \
0             NaN  ...   NaN             NaN   NaN             NaN   NaN   
1     

In [9]:
# Define a dictionary to rename columns
rename_dict = {
    "DATE": "Date",
    "AWND": "Avg_Wind_Speed",
    "PRCP": "Precipitation",
    "SNOW": "Snowfall",
    "SNWD": "Snow_Depth",
    "TAVG": "Avg_Temperature",
    "TMAX": "Max_Temperature",
    "TMIN": "Min_Temperature",
    "WT01": "Fog",
    "WT02": "Heavy_Fog",
    "WT03": "Thunder",
    "WT04": "Ice_Pellets",
    "WT05": "Hail",
    "WT06": "Glaze_Ice",
}

# Apply renaming to the dataframe
df_ord = df_ord.rename(columns=rename_dict)

In [11]:
df["CRASH_DATE"] = pd.to_datetime(df["CRASH_DATE"])  # From crash dataset
df_ord["Date"] = pd.to_datetime(df_ord["Date"])  # From weather dataset

In [17]:
# Merge crash data with weather data on the date
df_merged = pd.merge(df, df_ord, left_on="CRASH_DATE", right_on="Date", how="left")

# Drop redundant Date column (since we already have CRASH_DATE)
df_merged.drop(columns=["Date"], inplace=True)

# Display first few rows
df_merged.head(10)



#df_merged.info()  # Check for missing values after merging



,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,Ice_Pellets,WT04_ATTRIBUTES,Hail,WT05_ATTRIBUTES,Glaze_Ice,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT09,WT09_ATTRIBUTES
0,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,NaN,2018-01-01,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,Y,2018-01-01,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,NaN,2018-01-01,25,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dfb3229940c672426c4010d7348d16b69b030191213b86...,Y,2018-01-01,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,821ee3270b2c503fd0905415f674d3f4a63e1ac15d9100...,NaN,2018-01-01,35,NO CONTROLS,FUNCTIONING PROPERLY,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,610c39ab9557f7a90405c9b2db8dd20fa25f870f735f1e...,NaN,2018-01-01,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ea52144c44e6c2841ea4fa0050dc9220ed7b5a391f9f47...,NaN,2018-01-01,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DARKNESS,REAR END,OTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,e1f23baa35fc1e6e90859ac2555286e894f3cf49af84ee...,NaN,2018-01-01,30,TRAFFIC SIGNAL,FUNCTIONING IMPROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16fee228936a413e88bf55aeef16467efe4098d6082ca0...,NaN,2018-01-01,35,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0f197c975e852ec4df195054f2aa27da333049152d4cd8...,NaN,2018-01-01,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_merged.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768734 entries, 0 to 768733
Data columns (total 100 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   CRASH_RECORD_ID                768734 non-null  object        
 1   CRASH_DATE_EST_I               55503 non-null   object        
 2   CRASH_DATE                     768734 non-null  datetime64[ns]
 3   POSTED_SPEED_LIMIT             768734 non-null  int64         
 4   TRAFFIC_CONTROL_DEVICE         768734 non-null  object        
 5   DEVICE_CONDITION               768734 non-null  object        
 6   WEATHER_CONDITION              768734 non-null  object        
 7   LIGHTING_CONDITION             768734 non-null  object        
 8   FIRST_CRASH_TYPE               768734 non-null  object        
 9   TRAFFICWAY_TYPE                768734 non-null  object        
 10  LANE_CNT                       91692 non-null   float64       
 11 

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Aggregate crashes to daily totals
df_daily = df_merged.groupby("CRASH_DATE").agg({
    "CRASH_RECORD_ID": "count",  # Count number of crashes per day
    "Precipitation": "mean",  # Average precipitation
    "Avg_Temperature": "mean"  # Average temperature
}).reset_index()

# Set Date as Index
df_daily = df_daily.set_index("CRASH_DATE")


In [ ]:
# Apply Holt-Winters smoothing with a 7-day seasonal cycle
crash_model = ExponentialSmoothing(df_daily["Total_Crashes"],
                                   trend="add",  # Trend changes over time
                                   seasonal="add",  # Capture seasonality
                                   seasonal_periods=7)  # Weekly cycle
crash_fit = crash_model.fit()

# Save fitted values
df_daily["HWS_Crashes"] = crash_fit.fittedvalues

# Forecast next 30 days
forecast_crashes = crash_fit.forecast(steps=30)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(df_daily.index, df_daily["Total_Crashes"], alpha=0.5, label="Observed Crashes")
plt.plot(df_daily.index, df_daily["HWS_Crashes"], color="red", linestyle="--", label="HWS Fitted Values")
plt.plot(pd.date_range(df_daily.index[-1], periods=30, freq="D"), forecast_crashes, color="green", linestyle="--", label="HWS Forecast (30 Days)")
plt.xlabel("Date")
plt.ylabel("Total Crashes")
plt.title("Holt-Winters Smoothing: Total Crashes (7-Day Seasonality)")
plt.legend()
plt.grid(True)
plt.show()


Edit more here.


In [1]:
print(df_daily.head(10))

NameError: name 'df_daily' is not defined